In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import ElasticNetCV, RidgeCV, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
from sklearn.base import BaseEstimator, TransformerMixin

# Suppress warnings
warnings.filterwarnings("ignore")

# --- Custom transformer for clipping ---
class PercentileClipper(BaseEstimator, TransformerMixin):
    """Clips features based on training data percentiles."""
    def __init__(self, lower_percentile=1.0, upper_percentile=99.0):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        self.lower_bounds_ = np.percentile(X, self.lower_percentile, axis=0)
        self.upper_bounds_ = np.percentile(X, self.upper_percentile, axis=0)
        return self

    def transform(self, X):
        X_clipped = np.clip(X, self.lower_bounds_, self.upper_bounds_)
        return X_clipped

# --- Feature Engineering Function ---
def feature_engineering(df, is_train=True):
    df = df.copy()
    df['Equipment_Volume'] = df['Equipment_Height'] * df['Equipment_Width']
    #df['Equipment_Value_Log'] = np.log1p(df['Equipment_Value'])
    
    return df
   

# --- Load data ---
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')

# --- Extract target first ---
# --- Keep target ---
df_train = df_train[df_train['Transport_Cost'] >= 0]
df_train['Log_Transport_Cost'] = np.log1p(df_train['Transport_Cost'])
y_train_full = df_train['Log_Transport_Cost'] # Renamed to y_train_full for clarity

# --- Apply feature engineering on full train/test (do NOT drop date/ID columns yet) ---
df_train_fe= feature_engineering(df_train, is_train=True)
df_test_fe = feature_engineering(df_test, is_train=False)

# --- Drop non-feature columns AFTER feature engineering ---
drop_cols_train = ['Transport_Cost', 'Log_Transport_Cost', 'Hospital_Id',  
                   'Order_Placed_Date', 'Delivery_Date', 'Hospital_Location', 'Supplier_Name',
                   'Equipment_Height', 'Equipment_Width'] 
drop_cols_test = ['Hospital_Id', 'Order_Placed_Date', 'Delivery_Date', 'Hospital_Location', 'Supplier_Name',
                  'Equipment_Height', 'Equipment_Width']

X_train_full = df_train_fe.drop(columns=[c for c in drop_cols_train if c in df_train_fe.columns])
X_test       = df_test_fe.drop(columns=[c for c in drop_cols_test if c in df_test_fe.columns])

# --- Feature lists ---
numeric_features = [
    'Supplier_Reliability', 
    'Base_Transport_Fee',
    'Equipment_Volume', 
    'Equipment_Weight',
    'Equipment_Value', 
]


categorical_features = [
    'Equipment_Type',
    'CrossBorder_Shipping',
    'Installation_Service', 
    'Transport_Method',
    'Fragile_Equipment', 
    'Hospital_Info',
    'Rural_Hospital'
]

# --- Preprocessing pipelines ---
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    PercentileClipper(lower_percentile=1.0, upper_percentile=99.0),
    StandardScaler()
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore', sparse_output=False)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop' 
)


# --- Train/validation split ---
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)


# --- Fit preprocessor on training set, transform val/test ---
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed   = preprocessor.transform(X_val)

# X_test is the DataFrame with the correct columns (non-features dropped)
X_test_processed  = preprocessor.transform(X_test) 

print(f"Training features shape: {X_train_processed.shape}")
print(f"Validation features shape: {X_val_processed.shape}")
print(f"Test features shape: {X_test_processed.shape}")

Training features shape: (3605, 25)
Validation features shape: (902, 25)
Test features shape: (500, 25)


# Linear Regression without Regularization

In [35]:
######### linear regression without regularization #########
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# selectKBest features was tried but got better results without it so
from sklearn.linear_model import LinearRegression  # <-- We import the normal one

print("--- Starting Model 1: Normal Linear Regression ---")

lr_model = LinearRegression()
lr_model.fit(X_train_processed, y_train)

print("Model 1 (Linear Regression) trained.")

# --- Evaluate Model 1 ---
y_pred_log = lr_model.predict(X_val_processed)

if np.any(y_pred_log > 700):
    print("WARNING: Elastic Net produced an unstable (infinite) prediction. Model failed.")
else:
    # Get predictions on the TRAINING data (log-scale) to calculate residuals
    y_train_pred_log = lr_model.predict(X_train_processed)
    # Calculate the log-scale residuals (errors)
    train_residuals_log = y_train - y_train_pred_log

    y_val_actual = np.expm1(y_val)
    y_pred_actual = np.expm1(y_pred_log)
    y_pred_actual_corrected = y_pred_actual
    
    print("\n--- Model 1 Evaluation ---")

    rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual_corrected))
    r2 = r2_score(y_val_actual, y_pred_actual_corrected)
    mae = mean_absolute_error(y_val_actual, y_pred_actual_corrected)

    print(f"Validation RMSE: ${rmse:,.2f}")
    print(f"Validation R-squared: {r2:.3f}")
    print(f"Validation MAE: ${mae:,.3f}")

    # on log valeus
    rmse_log = np.sqrt(mean_squared_error(y_val, y_pred_log))
    r2_log = r2_score(y_val, y_pred_log)
    mae_log = mean_absolute_error(y_val, y_pred_log)
    
    print(f"Validation RMSE (log-scale): {rmse_log:.3f}")
    print(f"Validation R-squared (log-scale): {r2_log:.3f}")
    print(f"Validation MAE (log-scale): {mae_log:.3f}")

    y_pred_log = lr_model.predict(X_test_processed)
    y_pred_actual = np.expm1(y_pred_log)
    
    # Ensure all predictions are non-negative for submission
    y_pred_actual = np.maximum(0, y_pred_actual) 
    
    # The 'y_pred_actual_corrected' variable is now just the simple inverse transform, 
    # as with bias correction the result was worse 
    y_pred_actual_final = y_pred_actual 

    # --- Create submission file ---
    submission_df = pd.DataFrame({
        'Hospital_Id': df_test['Hospital_Id'],
        'Transport_Cost': y_pred_actual_final 
    })
    submission_df.to_csv('submission_linear_normal.csv', index=False)
    print("\n--- submission_linear_normal.csv created successfully! ---")
    print(submission_df.head())


--- Starting Model 1: Normal Linear Regression ---
Model 1 (Linear Regression) trained.

--- Model 1 Evaluation ---
Validation RMSE: $105,378.64
Validation R-squared: 0.462
Validation MAE: $12,486.827
Validation RMSE (log-scale): 0.747
Validation R-squared (log-scale): 0.774
Validation MAE (log-scale): 0.472

--- submission_linear_normal.csv created successfully! ---
            Hospital_Id  Transport_Cost
0          fffe33003400      611.057394
1  fffe3700330036003600      281.620369
2  fffe3300390038003400     3969.136283
3      fffe310030003900      303.932761
4  fffe3700330031003200      812.867667


# Linear Regression -- LASSO Regularization

In [36]:
# ---  Train Lasso Model ---
# LassoCV will find the best alpha
lasso_model = LassoCV(
    alphas=np.logspace(-5, -1, 50), # 50 alphas between 10^-5 and 10^-1
    cv=5,
    max_iter=10000,
    random_state=42,
    n_jobs=-1
)
lasso_model.fit(X_train_processed, y_train)

print("Model 2 (Lasso) trained.")
print("-" * 50)

# --- Evaluate Model 2 ---
y_pred_log = lasso_model.predict(X_val_processed)

if np.any(y_pred_log > 700):
    print("WARNING: Lasso produced an unstable (infinite) prediction. Model failed.")
else:
    # Get predictions on the TRAINING data (log-scale) to calculate residuals
    y_train_pred_log = lasso_model.predict(X_train_processed)
    # Calculate the log-scale residuals (errors)
    train_residuals_log = y_train - y_train_pred_log

    y_val_actual = np.expm1(y_val)
    y_pred_actual = np.expm1(y_pred_log)
    # Set corrected = actual (no correction factor)
    y_pred_actual_corrected = y_pred_actual 

    print("\n--- Model 2 Evaluation ---")
    rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual_corrected))
    r2 = r2_score(y_val_actual, y_pred_actual_corrected)
    mae = mean_absolute_error(y_val_actual, y_pred_actual_corrected)

    print(f"Validation RMSE: ${rmse:,.2f}")
    print(f"Validation R-squared: {r2:.3f}")
    print(f"Validation MAE: ${mae:,.3f}")
    print(f"Best Alpha selected by LassoCV: {lasso_model.alpha_}")

    # --- Count dropped (zero) coefficients ---
    n_total_features = X_train_processed.shape[1]
    n_zero_coeff = np.sum(lasso_model.coef_ == 0)
    n_nonzero_coeff = n_total_features - n_zero_coeff
    
    print(f"Total features: {n_total_features}")
    print(f"Features dropped (zero coefficients): {n_zero_coeff}")
    print(f"Features retained: {n_nonzero_coeff}")

    print("\n--- Log-Scale Evaluation (Model Fit) ---")
    rmse_log = np.sqrt(mean_squared_error(y_val, y_pred_log))
    r2_log = r2_score(y_val, y_pred_log)
    mae_log = mean_absolute_error(y_val, y_pred_log)

    print(f"Validation RMSE (log-scale): {rmse_log:.3f}")
    print(f"Validation R-squared (log-scale): {r2_log:.3f}")
    print(f"Validation MAE (log-scale): {mae_log:.3f}")

    # --- 3. Create Submission File (Lasso) ---
    y_pred_log_test = lasso_model.predict(X_test_processed)
    y_pred_actual_test = np.expm1(y_pred_log_test)
    
    # Ensure all predictions are non-negative for submission
    y_pred_actual_test = np.maximum(0, y_pred_actual_test) 
    y_pred_actual_final = y_pred_actual_test 

    submission_df = pd.DataFrame({
        'Hospital_Id': df_test['Hospital_Id'],
        'Transport_Cost': y_pred_actual_final
    })
    submission_df.to_csv('submission_lasso.csv', index=False)
    print("\n--- submission_lasso.csv created successfully! ---")
    print(submission_df.head())

Model 2 (Lasso) trained.
--------------------------------------------------

--- Model 2 Evaluation ---
Validation RMSE: $105,082.14
Validation R-squared: 0.465
Validation MAE: $12,445.939
Best Alpha selected by LassoCV: 0.00029470517025518097
Total features: 25
Features dropped (zero coefficients): 3
Features retained: 22

--- Log-Scale Evaluation (Model Fit) ---
Validation RMSE (log-scale): 0.747
Validation R-squared (log-scale): 0.774
Validation MAE (log-scale): 0.472

--- submission_lasso.csv created successfully! ---
            Hospital_Id  Transport_Cost
0          fffe33003400      611.455857
1  fffe3700330036003600      282.464141
2  fffe3300390038003400     3958.812378
3      fffe310030003900      303.014406
4  fffe3700330031003200      812.707942


# Linear Regression -- Ridge Regularization

In [37]:
# --- 1. Train Ridge Model ---
# RidgeCV will find the best alpha. Ridge is often more stable
# and efficient, so we can test a wider range of alphas.
ridge_model = RidgeCV(
    alphas=np.logspace(-3, 3, 50), # 50 alphas between 10^-3 and 10^3
    cv=5
)
ridge_model.fit(X_train_processed, y_train)

print("Model 3 (Ridge) trained.")
print("-" * 50)

# --- Evaluate Model 3 ---
y_pred_log = ridge_model.predict(X_val_processed)

if np.any(y_pred_log > 700):
    print("WARNING: Ridge produced an unstable (infinite) prediction. Model failed.")
else:
    # Get predictions on the TRAINING data (log-scale) to calculate residuals
    y_train_pred_log = ridge_model.predict(X_train_processed)
    # Calculate the log-scale residuals (errors)
    train_residuals_log = y_train - y_train_pred_log
    
    y_val_actual = np.expm1(y_val)
    y_pred_actual = np.expm1(y_pred_log)
    # Set corrected = actual (no correction factor)
    y_pred_actual_corrected = y_pred_actual 

    print("\n--- Model 3 Evaluation ---")
    rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual_corrected))
    r2 = r2_score(y_val_actual, y_pred_actual_corrected)
    mae = mean_absolute_error(y_val_actual, y_pred_actual_corrected)

    print(f"Validation RMSE: ${rmse:,.2f}")
    print(f"Validation R-squared: {r2:.3f}")
    print(f"Validation MAE: ${mae:,.3f}")
    print(f"Best Alpha selected by RidgeCV: {ridge_model.alpha_}")

    print("\n--- Log-Scale Evaluation (Model Fit) ---")
    rmse_log = np.sqrt(mean_squared_error(y_val, y_pred_log))
    r2_log = r2_score(y_val, y_pred_log)
    mae_log = mean_absolute_error(y_val, y_pred_log)

    print(f"Validation RMSE (log-scale): {rmse_log:.3f}")
    print(f"Validation R-squared (log-scale): {r2_log:.3f}")
    print(f"Validation MAE (log-scale): {mae_log:.3f}")

    # --- Create Submission File ---
    y_pred_log_test = ridge_model.predict(X_test_processed)
    y_pred_actual_test = np.expm1(y_pred_log_test)
    
    # Ensure all predictions are non-negative for submission
    y_pred_actual_test = np.maximum(0, y_pred_actual_test) 
    y_pred_actual_final = y_pred_actual_test 

    submission_df = pd.DataFrame({
        'Hospital_Id': df_test['Hospital_Id'],
        'Transport_Cost': y_pred_actual_final
    })
    submission_df.to_csv('submission_ridge.csv', index=False)
    print("\n--- submission_ridge.csv created successfully! ---")
    print(submission_df.head())

Model 3 (Ridge) trained.
--------------------------------------------------

--- Model 3 Evaluation ---
Validation RMSE: $104,974.19
Validation R-squared: 0.467
Validation MAE: $12,434.654
Best Alpha selected by RidgeCV: 6.25055192527397

--- Log-Scale Evaluation (Model Fit) ---
Validation RMSE (log-scale): 0.747
Validation R-squared (log-scale): 0.774
Validation MAE (log-scale): 0.472

--- submission_ridge.csv created successfully! ---
            Hospital_Id  Transport_Cost
0          fffe33003400      613.231112
1  fffe3700330036003600      283.283091
2  fffe3300390038003400     3922.995097
3      fffe310030003900      303.675635
4  fffe3700330031003200      814.697890


# Linear Regression -- Elastic Net Regularization

In [38]:
# --- Train Elastic Net Model ---
# We give it a list of L1 ratios (the mix) and it will find the best alpha
elastic_model = ElasticNetCV(
    # Narrow the search around 1.0 (Lasso) since the previous model chose 1.0
    l1_ratio=[.9, .95, .99, .999, 1.0], 
    alphas=np.logspace(-4, -1, 30), # Increase alpha search points
    cv=5,
    max_iter=10000, # Increased max_iter for stability
    random_state=42,
    n_jobs=-1
)
elastic_model.fit(X_train_processed, y_train)

print("Model 4 (Elastic Net) trained.")
print("-" * 50)

# --- 8. Evaluate Model 4 ---
y_pred_log = elastic_model.predict(X_val_processed)

if np.any(y_pred_log > 700):
    print("WARNING: Elastic Net produced an unstable (infinite) prediction. Model failed.")
else:
    # Get predictions on the TRAINING data (log-scale) to calculate residuals
    y_train_pred_log = elastic_model.predict(X_train_processed)
    # Calculate the log-scale residuals (errors)
    train_residuals_log = y_train - y_train_pred_log

    y_val_actual = np.expm1(y_val)
    y_pred_actual = np.expm1(y_pred_log)
    y_pred_actual_corrected = y_pred_actual

    print("\n--- Model 4 Evaluation ---")

    rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual_corrected))
    r2 = r2_score(y_val_actual, y_pred_actual_corrected)
    mae = mean_absolute_error(y_val_actual, y_pred_actual_corrected)

    print(f"Validation RMSE: ${rmse:,.2f}")
    print(f"Validation R-squared: {r2:.3f}")
    print(f"Validation MAE: ${mae:,.3f}")
    print(f"Best L1 Ratio selected by ElasticNetCV: {elastic_model.l1_ratio_}")
    print(f"Best Alpha selected by ElasticNetCV: {elastic_model.alpha_}")

    print("\n--- Log-Scale Evaluation (Model Fit) ---")
    rmse_log = np.sqrt(mean_squared_error(y_val, y_pred_log))
    r2_log = r2_score(y_val, y_pred_log)
    mae_log = mean_absolute_error(y_val, y_pred_log)

    print(f"Validation RMSE (log-scale): {rmse_log:.3f}")
    print(f"Validation R-squared (log-scale): {r2_log:.3f}")
    print(f"Validation MAE (log-scale): {mae_log:.3f}")

    y_pred_log = elastic_model.predict(X_test_processed)
    y_pred_actual = np.expm1(y_pred_log)
    
    # Ensure all predictions are non-negative for submission
    y_pred_actual = np.maximum(0, y_pred_actual) 
    y_pred_actual_final = y_pred_actual 
    
    # --- Create submission file ---
    submission_df = pd.DataFrame({
        'Hospital_Id': df_test['Hospital_Id'],
        'Transport_Cost': y_pred_actual_final 
    })
    submission_df.to_csv('submission_elastic.csv', index=False)
    print("\n--- submission_elastic.csv created successfully! ---")
    print(submission_df.head())


Model 4 (Elastic Net) trained.
--------------------------------------------------

--- Model 4 Evaluation ---
Validation RMSE: $105,077.49
Validation R-squared: 0.465
Validation MAE: $12,445.386
Best L1 Ratio selected by ElasticNetCV: 0.9
Best Alpha selected by ElasticNetCV: 0.00032903445623126676

--- Log-Scale Evaluation (Model Fit) ---
Validation RMSE (log-scale): 0.747
Validation R-squared (log-scale): 0.774
Validation MAE (log-scale): 0.472

--- submission_elastic.csv created successfully! ---
            Hospital_Id  Transport_Cost
0          fffe33003400      611.481893
1  fffe3700330036003600      282.480102
2  fffe3300390038003400     3957.603469
3      fffe310030003900      302.990553
4  fffe3700330031003200      812.987080


# Polynomial Regression of degree 2 

In [39]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score

# --- 1. Polynomial Features (degree 2) ---
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train_processed)

X_train_poly = poly.transform(X_train_processed)
X_val_poly   = poly.transform(X_val_processed)
X_test_poly  = poly.transform(X_test_processed)

print(f"Polynomial features created. New shape: {X_train_poly.shape}")

# --- 2. Re-Scale the Polynomial Features ---
scaler_poly = StandardScaler()
X_train_poly_scaled = scaler_poly.fit_transform(X_train_poly)
X_val_poly_scaled   = scaler_poly.transform(X_val_poly)
X_test_poly_scaled  = scaler_poly.transform(X_test_poly)

print("Polynomial features scaled.")

# --- 3. Train Ridge (L2) Model on Poly Features ---
poly_ridge_model = RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0], cv=5)
poly_ridge_model.fit(X_train_poly_scaled, y_train)

print("Model 5 (Polynomial + Ridge) trained.")

# --- 4. Evaluate on Validation Set ---
y_pred_log = poly_ridge_model.predict(X_val_poly_scaled)

if np.any(np.isnan(y_pred_log)) or np.any(np.isinf(y_pred_log)):
    print("WARNING: Polynomial model produced unstable predictions.")
else:
    # Inverse-transform (log → actual)
    y_val_actual = np.expm1(y_val)
    y_pred_actual = np.expm1(y_pred_log)

    # Metrics on actual values
    rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual))
    r2 = r2_score(y_val_actual, y_pred_actual)
    mae = mean_absolute_error(y_val_actual, y_pred_actual)

    print("\n--- Model 5 Evaluation (Actual Scale) ---")
    print(f"Validation RMSE: ${rmse:,.2f}")
    print(f"Validation R-squared: {r2:.3f}")
    print(f"Validation MAE: ${mae:,.2f}")
    print(f"Best Alpha selected by RidgeCV: {poly_ridge_model.alpha_}")

    # Metrics on log scale
    rmse_log = np.sqrt(mean_squared_error(y_val, y_pred_log))
    r2_log = r2_score(y_val, y_pred_log)
    mae_log = mean_absolute_error(y_val, y_pred_log)

    print("\n--- Model 5 Evaluation (Log Scale) ---")
    print(f"Validation RMSE (log): {rmse_log:.3f}")
    print(f"Validation R² (log): {r2_log:.3f}")
    print(f"Validation MAE (log): {mae_log:.3f}")

    # Cross-validation on training data
    scores = cross_val_score(poly_ridge_model, X_train_poly_scaled, y_train, scoring='r2', cv=5)
    print("\nAverage R² (5-fold CV):", scores.mean())

    # --- 5. Predict on Test Data ---
    y_pred_log_test = poly_ridge_model.predict(X_test_poly_scaled)
    y_pred_actual_test = np.expm1(y_pred_log_test)

    # Retrieve Hospital_Id from df_test
    test_ids = df_test["Hospital_Id"]

    # Create submission file
    submission_df = pd.DataFrame({
        'Hospital_Id': test_ids,
        'Transport_Cost': y_pred_actual_test
    })

    submission_df.to_csv('submission_poly2.csv', index=False)
    print("\n--- submission_poly2.csv created successfully! ---")
    print(submission_df.head())


Polynomial features created. New shape: (3605, 350)
Polynomial features scaled.
Model 5 (Polynomial + Ridge) trained.

--- Model 5 Evaluation (Actual Scale) ---
Validation RMSE: $236,274.06
Validation R-squared: -1.703
Validation MAE: $24,188.17
Best Alpha selected by RidgeCV: 100.0

--- Model 5 Evaluation (Log Scale) ---
Validation RMSE (log): 0.610
Validation R² (log): 0.850
Validation MAE (log): 0.351

Average R² (5-fold CV): 0.8772245576593537

--- submission_poly2.csv created successfully! ---
            Hospital_Id  Transport_Cost
0          fffe33003400      590.664047
1  fffe3700330036003600      230.085863
2  fffe3300390038003400     4236.898356
3      fffe310030003900      241.965815
4  fffe3700330031003200     1110.902280


# for polynomial of degree 3:
# By creating so many extra features, the model became so complex and unstable that its predictions on new data are completely nonsensical, resulting in a gigantic error. It is better to avoid training it and stick to max, a polynomial of degree 2 